In [36]:
import re
import time
import requests
import tldextract
from bs4 import BeautifulSoup
import cchardet
import traceback
import leveldb

In [13]:
def save_to_db(url,html):
    print('{url} : {num}'.format(url=url,num=len(html)))

In [16]:
def crawl():
    url = 'http://news.baidu.com/'
    res = requests.get(url)
    html = res.text
    bf = BeautifulSoup(html)
    
    links = re.findall(r'href=[\'"]?(.*?)[\'"\s]',html)
    print('find links:',len(links))
    news_links = []
    
    for link in links:
        if not link.startswith('http'):
            continue
        tld = tldextract.extract(link)
        if tld.domain == 'baidu':
            continue
        news_links.append(link)
    print('find news links:',len(news_links))
    
    for link in news_links:
        html = requests.get(link).text
        save_to_db(link,html)
    print('works done!')

In [19]:
def main():
    while 1:
        crawl()
        time.sleep(10)

In [32]:
def downloader(url,timeout=10,headers=None,debug=False,binary=False):
    _headers = {
        "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36',
        
    }
    redirected_url = url
    if headers:
        _headers = headers
    try:
        r = requests.get(url,headers=_headers,timeout=timeout)
        if binary:
            html = r.content
        else:
            encoding = cchardet.detect(r.content)['encodig']
            html = r.content.decode(encoding)
        status = r.status_code
        redirected_url = r.url
    except:
        if debug:
            traceback.print_exc()
        msg = 'failed download: {}'.format(url)
        print(msg)
        if binary:
            html = b''
        else:
            html = ''
        status = 0
    return status,html,redirected_url

In [33]:
g_bin_postfix = set([
    'exe', 'doc', 'docx', 'xls', 'xlsx', 'ppt', 'pptx',
    'pdf',
    'jpg', 'png', 'bmp', 'jpeg', 'gif',
    'zip', 'rar', 'tar', 'bz2', '7z', 'gz',
    'flv', 'mp4', 'avi', 'wmv', 'mkv',
    'apk',
])

g_news_postfix = [
    '.html?', '.htm?', '.shtml?',
    '.shtm?',
]


def clean_url(url):
    # 1. 是否为合法的http url
    if not url.startswith('http'):
        return ''
    # 2. 去掉静态化url后面的参数
    for np in g_news_postfix:
        p = url.find(np)
        if p > -1:
            p = url.find('?')
            url = url[:p]
            return url
    # 3. 不下载二进制类内容的链接
    up = urlparse.urlparse(url)
    path = up.path
    if not path:
        path = '/'
    postfix = path.split('.')[-1].lower()
    if postfix in g_bin_postfix:
        return ''

    # 4. 去掉标识流量来源的参数
    # badquery = ['spm', 'utm_source', 'utm_source', 'utm_medium', 'utm_campaign']
    good_queries = []
    for query in up.query.split('&'):
        qv = query.split('=')
        if qv[0].startswith('spm') or qv[0].startswith('utm_'):
            continue
        if len(qv) == 1:
            continue
        good_queries.append(query)
    query = '&'.join(good_queries)
    url = urlparse.urlunparse((
        up.scheme,
        up.netloc,
        path,
        up.params,
        query,
        ''  #  crawler do not care fragment
    ))
    return url

In [37]:
class UrlDB(object):
    status_failure = b'0'
    status_success = b'1'
    
    def __init__(self,db_name):
        self.name = db_name + '.urldb'
        self.db = leveldb.LevelDB(self.name)
        
    def set_success(self,url):
        if isinstance(url,str):
            url = url.encode('utf8')
        try:
            self.db.Put(url,self.status_success)
            s = True
        except:
            s = False
        return s
    
    def set_failure(self,url):
        if isinstance(url,str):
            url = url.encode('utf8')
        try:
            self.db.Put(url,self.status_failure)
            s = True
        except:
            s = False
        return s
    
    def has(self,url):
        if isinstance(url,str):
            url = url.encode('utf8')
        try:
            attr = self.db.Get(url)
            return attr
        except:
            pass
        return False